- Configure a Caching DNS Server
  - `bind` and `bind-utils` are the two packages to install
  - The config file for bind is `/etc/named.conf`
  - In `named.conf`, you need to configure the following directives:
    - `listen-on port 53`. This needs to include the IP address of a NIC on the server. This tells the server what IP address to listen for queries on.
      - Example:
        ```
        //
        // named.conf
        //
        // Provided by Red Hat bind package to configure the ISC BIND named(8) DNS
        // server as a caching only nameserver (as a localhost DNS resolver only).
        //
        // See /usr/share/doc/bind*/sample/ for example named configuration files.
        //

        options {
                listen-on port 53 { 127.0.0.1; 172.16.1.4; };
                listen-on-v6 port 53 { ::1; };
                directory       "/var/named";
                dump-file       "/var/named/data/cache_dump.db";
                statistics-file "/var/named/data/named_stats.txt";
                memstatistics-file "/var/named/data/named_mem_stats.txt";
                secroots-file   "/var/named/data/named.secroots";
                recursing-file  "/var/named/data/named.recursing";
                allow-query     { localhost; 172.16.1.0/24; };
        ```
    - `allow-query`. This tells the server where queries are allowed to originate from. You can add a subnet or `any;` as a value
      - Example:
        ```
                allow-query     { localhost; 172.16.1.0/24; any; };
        ```
- Maintain a DNS Zone
  - To tell bind it is responsible for a zone, edit `/etc/named.conf` and add:
    ```
    zone "example.com." IN {
        type master;
        file "example.com.zone";
    };
    ```
    - Next, create a file named `example.com.zone` in `/var/named`. BIND stores zone files in `/var/named`
      - You can actually copy the `named.localhost` file to a new zone file
        `sudo cp --preserve=ownership /var/named/named.localhost /var/named/example.com.zone`

- Configure Email Aliases
  - Install `postfix`
  - email aliases are defined in `/etc/aliases`
  - After adding a new alias to `/etc/aliases`, you must run the command `newaliases` to tell the mail daemon there are new aliases
  - Configure IMAP/IMAPs
    - Install `dovecot`
    - Config file: `/etc/dovecot/dovecot.conf`
      - More config files are in `/etc/dovecot/conf.d/`
  - To send email via the command line to john@example.com:
    - `echo "This is the body" | mailx -s "This is the subject" john@example.com

- Configure SSH Servers and Clients
  - Config file: `/etc/ssh/sshd_config`
    - Commonly configured values:
      - `PermitRootLogin`
      - `Port`, `AddressFamily`, `ListenAddresses`
      - `PasswordAuthentication`
      - Overriding settings on a per user basis:
        ```
        # Example of overriding settings on a per-user basis
        #Match User anoncvs
        #       X11Forwarding no
        #       AllowTcpForwarding no
        #       PermitTTY no
        #       ForceCommand cvs server
        #       PasswordAuthentication yes
        ```
  - Example ~/.ssh/config file:
    ```
    Host centos
        HostName 172.16.1.4
        Port 22
        User azureadmin
    ```
  - To authorize users to connect via ssh to a server, add their public key to ~/.ssh/authorized_keys

- Restrict Access to the HTTP Proxy Server
  - Squid is a well known proxy for Linux
  - `dnf install squid -y`
  - Add a rule to firewalld after starting/enabling the service
    - `sudo firewall-cmd --add-service=squid`
  -  `/etc/squid/squid.conf` contains ACLs
     -  deny rules should be processed last

- Configure an HTTP Server
  - Install Apache: `sudo dnf install httpd -y`
  - Enable firewall ports: `sudo firewall-cmd --add-service=http`, `sudo firewall-cmd --add-service=https`, `sudo firewall-cmd --runtime-to-permanent`
  - to use https, install mod-ssl: `sudo dnf install mod_ssl -y`
  - Generate TLS Certs:
  - Start/enable httpd
  - Config files are in `/etc/httpd`
  - Create VirtualHost Files in `/etc/httpd/conf.d/`
    - Example VirtualHost file:
      ```
      <VirtualHost *:80>
        ServerName store.example.com
        DocumentRoot /var/www/store
      </VirtualHost>

      <VirtualHost *:80>
              ServerName blog.example.com
              DocumentRoot /var/www/blog/
      </VirtualHost>
      ```
  - SSL Settings File: `/etc/ssh/conf.d/ssl.conf`
  - Modules are listed in `/etc/httpd/conf.modules.d/00-base.conf`

- Configure HTTP Server Log Files
  - httpd stores its main logs in two files:
    - `error_log`: stores errors in /etc/httpd/logs/error_log
    - `access_log`: stores access related messages
      - Can be configured using the custom log directives
    - `/etc/httpd/` in the paths above is configured via the ServerRoot directive in `/etc/httpd/conf/httpd.conf`
    - `LogLevel` directive in `/etc/httpd/conf/httpd.conf` controls the log level and is set to `warn` by default
    - You can add logging for a specific virtual host in the VirtualHost file. VirtualHost files are stored in `/etc/httpd/conf.d/`
      ```
      <VirtualHost *:80>
              ServerName store.example.com
              DocumentRoot /var/www/store/
              CustomLog /var/log/httpd/store.example.com_access.log combined
              ErrorLog /var/log/httpd/store.example.com_error.log
      </VirtualHost>
      ```
- Restrict Access to a Web Page
  - In `/etc/httpd/conf/httpd.conf`, configure the `Options` directive. 
    ```
    # Indexes Includes FollowSymLinks SymLinksifOwnerMatch ExecCGI MultiViews
    # Options Indexes FollowSymLinks
    ```

- Configure a Database Server
  - Install MariaDb: 
    - `sudo dnf install mariadb-server -y`
  - Start the service, enable it
  - Allow access through the firewall
    - `sudo firewall-cmd --add-service=mysql`
    - `sudo firewall-cmd --runtime-to-permanent`
  - To interact with the database
    - `mysql -u root`
  - To secure the installation:
    - mysql_secure_installation
  - Important config files:
    - `/etc/my.cnf`
    - `/etc/my.conf.d/mariadb-server.cnf`

- Manage and Configure Containers
  - Install podman
    - `sudo dnf install docker -y`
    - This actually installs podman in CentOS

- Manage and Configure a Virtual Machine
  - Install libvirt and qemu-kvm
    - `sudo dnf install libvirt qemu-kvm -y`
  - VMs are XML files
    - `nano testmachine.xml`
      ```
      <domain type="qemu">
        <name>TestMachine</name>
        <memory unit="GiB">1</memory>
        <vcpu>1</vcpu>
        <os>
                <type arch='x86_64'>hvm</type>
        </os>
      </domain>
      ```
    - Create the VM from the XML file:
      - `virsh define testmachine.xml`
    - Destroy a VM, but leave its resources behind:
      - `virsh destroy VM1`
    - Destroy a VM and remove its resources:
      - `virsh destroy VM1`
      - `virsh undefine VM1`
    - List all domains:
      - `virsh --list`
    - Start a domain:
      - `sudo virsh start VM1`
    - Configure a domain to autostart:
      - `sudo virsh autostart --domain VM2`


- Commands:
```
dig

newaliases
sendmail

ssh
ssh-keygen
ssh-copy-id
```